# **Asteroid Prediction**


# Import Python Libraries

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Reading the Income Classification Dataset

In [ ]:
#%cd /content/drive/My Drive/Data

In [ ]:
df = pd.read_csv('/Asteroid.csv',  on_bad_lines='skip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,14,15,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()

,full_name,a,e,G,i,om,w,q,ad,per_y,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# Column Renaming

In [ ]:
readable_df = df.copy()
readable_df = readable_df.rename(columns={"a": "semi-major axis",
                            "e": "eccentricity",
                            "G": "Magnitude slope parameter",
                            "i": "Inclination with ecliptic plane",
                            "om": "Longitude of the ascending node",
                            "w": "argument of perihelion",
                            "q": "perihelion distance(au)",
                            "ad": "aphelion distance(au)",
                            "per_y": "Orbital period",
                            "data_arc": "data arc-span(d)",
                            "condition_code": "Orbit condition code",
                            "n_obs_used": "number of observations used",
                            "H": "Absolute Magnitude parameter",
                            "condition_code": "Orbit condition code",
                            })

In [ ]:
readable_df.head()

,full_name,semi-major axis,eccentricity,Magnitude slope parameter,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# **Exploration (EDA) & Visualization**

In [ ]:
readable_df.shape

(839736, 27)

In [ ]:
readable_df = readable_df.dropna(axis=0, subset=['diameter'])

In [ ]:
dtf_train, dtf_test = train_test_split(readable_df, test_size=0.2)

In [ ]:
dtf_train.isna().sum()

full_name                               0
semi-major axis                         0
eccentricity                            0
Magnitude slope parameter          110044
Inclination with ecliptic plane         0
Longitude of the ascending node         0
argument of perihelion                  0
perihelion distance(au)                 0
aphelion distance(au)                   0
Orbital period                          0
data arc-span(d)                      107
Orbit condition code                    0
number of observations used             0
Absolute Magnitude parameter          587
diameter                                0
extent                             110129
albedo                                981
rot_per                            101190
GM                                 110133
BV                                 109313
UB                                 109349
IR                                 110143
spec_B                             109035
spec_T                            

In [ ]:
dtf_train.dtypes

full_name                           object
semi-major axis                    float64
eccentricity                       float64
Magnitude slope parameter          float64
Inclination with ecliptic plane    float64
Longitude of the ascending node    float64
argument of perihelion             float64
perihelion distance(au)            float64
aphelion distance(au)              float64
Orbital period                     float64
data arc-span(d)                   float64
Orbit condition code                object
number of observations used          int64
Absolute Magnitude parameter       float64
diameter                            object
extent                              object
albedo                             float64
rot_per                            float64
GM                                 float64
BV                                 float64
UB                                 float64
IR                                 float64
spec_B                              object
spec_T     

In [ ]:
dtf_train.shape

(110144, 27)

In [ ]:
dtf_test.shape

(27537, 27)

In [ ]:
print(dtf_train.shape)
print(dtf_test.shape)

(110144, 27)
(27537, 27)


In [ ]:
# tf_train = dtf_train.dropna(axis=0, subset=['diameter'])
# tf_test = dtf_test.dropna(axis=0, subset=['diameter'])

In [ ]:
dtf_train['Orbit condition code'].value_counts()

0      100961
9        4532
0        1579
9.0       685
1         667
1         451
2         256
5         200
6         130
4         115
3         114
7         107
2          75
5.0        66
7.0        62
8          38
3.0        28
6.0        27
8.0        26
4.0        25
Name: Orbit condition code, dtype: int64

In [ ]:
dtf_train['Orbit condition code'].value_counts()

0      100961
9        4532
0        1579
9.0       685
1         667
1         451
2         256
5         200
6         130
4         115
3         114
7         107
2          75
5.0        66
7.0        62
8          38
3.0        28
6.0        27
8.0        26
4.0        25
Name: Orbit condition code, dtype: int64

In [ ]:
# dtf_train = dtf_train.drop(dtf_train[dtf_train.loc[:,'Orbit condition code']== 'E'].index)
# dtf_train = dtf_train.drop(dtf_train[dtf_train.loc[:,'Orbit condition code']== 'D'].index)

In [ ]:
dtf_train['moid'].value_counts()

1.097010    7
1.245500    7
1.160460    6
1.141000    6
1.253510    6
           ..
0.018117    1
0.702439    1
1.579470    1
1.751130    1
1.056090    1
Name: moid, Length: 81054, dtype: int64

In [ ]:
dtf_train = dtf_train.astype({'diameter': 'float64'})
dtf_test= dtf_test.astype({'diameter': 'float64'})

In [ ]:
categorical_variables = ['Orbit condition code', 'extent', 'spec_B', 'spec_T', 'neo', 'pha']

In [ ]:
nan_cols_con = ['semi-major axis', 'aphelion distance(au)', 'Magnitude slope parameter', 'data arc-span(d)', 'Absolute Magnitude parameter', 
                'albedo', 'rot_per', 'GM', 'BV', 'UB', 'IR', 'moid', 'Orbital period']

fill nan with mean value

In [ ]:
for arg in nan_cols_con:
  dtf_train[arg] = dtf_train[arg].fillna(dtf_train[arg].mean())
  dtf_test[arg] = dtf_test[arg].fillna(dtf_train[arg].mean())

In [ ]:
print(dtf_train.shape)
print(dtf_test.shape)

(110144, 27)
(27537, 27)


In [ ]:
dtf_train.isna().sum()

full_name                               0
semi-major axis                         0
eccentricity                            0
Magnitude slope parameter               0
Inclination with ecliptic plane         0
Longitude of the ascending node         0
argument of perihelion                  0
perihelion distance(au)                 0
aphelion distance(au)                   0
Orbital period                          0
data arc-span(d)                        0
Orbit condition code                    0
number of observations used             0
Absolute Magnitude parameter            0
diameter                                0
extent                             110129
albedo                                  0
rot_per                                 0
GM                                      0
BV                                      0
UB                                      0
IR                                      0
spec_B                             109035
spec_T                            

In [ ]:
dtf_train['pha'] = dtf_train['pha'].fillna(dtf_train['pha'].value_counts().index[0])
dtf_test['pha'] = dtf_test['pha'].fillna(dtf_train['pha'].value_counts().index[0])

In [ ]:
dtf_train['Orbit condition code'] = dtf_train['Orbit condition code'].fillna(dtf_train['Orbit condition code'].value_counts().index[0])
dtf_test['Orbit condition code'] = dtf_test['Orbit condition code'].fillna(dtf_train['Orbit condition code'].value_counts().index[0])

In [ ]:
for arg in ['extent', 'spec_B', 'spec_T', 'neo']:
  dtf_train[arg] = dtf_train[arg].fillna('U')
  dtf_test[arg] = dtf_test[arg].fillna('U')

In [ ]:
print(dtf_train.shape)
print(dtf_test.shape)

(110144, 27)
(27537, 27)


In [ ]:
dtf_train.isna().sum()

full_name                          0
semi-major axis                    0
eccentricity                       0
Magnitude slope parameter          0
Inclination with ecliptic plane    0
Longitude of the ascending node    0
argument of perihelion             0
perihelion distance(au)            0
aphelion distance(au)              0
Orbital period                     0
data arc-span(d)                   0
Orbit condition code               0
number of observations used        0
Absolute Magnitude parameter       0
diameter                           0
extent                             0
albedo                             0
rot_per                            0
GM                                 0
BV                                 0
UB                                 0
IR                                 0
spec_B                             0
spec_T                             0
neo                                0
pha                                0
moid                               0
d

In [ ]:
# cleaned_df = cleaned_df.drop(['full_name'], axis=1)
# cleaned_df.dtypes

In [ ]:
# real_diameter_df = cleaned_df.astype({'diameter': 'float64', "condition_code": "int32"})
# real_diameter_df.dtypes

In [ ]:
dtf_train = dtf_train.drop(['full_name'], axis=1)
dtf_test = dtf_test.drop(['full_name'], axis=1)

In [ ]:
dtf_test.isna().sum()

semi-major axis                    0
eccentricity                       0
Magnitude slope parameter          0
Inclination with ecliptic plane    0
Longitude of the ascending node    0
argument of perihelion             0
perihelion distance(au)            0
aphelion distance(au)              0
Orbital period                     0
data arc-span(d)                   0
Orbit condition code               0
number of observations used        0
Absolute Magnitude parameter       0
diameter                           0
extent                             0
albedo                             0
rot_per                            0
GM                                 0
BV                                 0
UB                                 0
IR                                 0
spec_B                             0
spec_T                             0
neo                                0
pha                                0
moid                               0
dtype: int64

## Dealing with Categorical variables
### neo and pha

In [ ]:
categorical_data = dtf_train.select_dtypes(include=['object']).copy()
categorical_data_test = dtf_test.select_dtypes(include=['object']).copy()
categorical_data 

,Orbit condition code,extent,spec_B,spec_T,neo,pha
115463,0,U,U,U,N,N
24218,0,U,U,U,N,N
459444,0,U,U,U,N,N
241880,0,U,U,U,N,N
53921,0,U,U,U,N,N
...,...,...,...,...,...,...
222749,0,U,U,U,N,N
222606,0,U,U,U,N,N
313623,0,U,U,U,N,N
88540,0,U,U,U,N,N


In [ ]:
dtf_train['Orbit condition code'].value_counts()

0      100961
9        4532
0        1579
9.0       685
1         667
1         451
2         256
5         200
6         130
4         115
3         114
7         107
2          75
5.0        66
7.0        62
8          38
3.0        28
6.0        27
8.0        26
4.0        25
Name: Orbit condition code, dtype: int64

In [ ]:
dtf_test['Orbit condition code'].value_counts()

0      25232
9       1147
0        438
9.0      187
1        142
1        112
2         46
5         42
4         29
5.0       26
7         22
6         22
3         21
2.0       20
7.0       16
8         11
6.0        9
4.0        8
3.0        5
8.0        2
Name: Orbit condition code, dtype: int64

In [ ]:
print(categorical_data.shape)
print(categorical_data_test.shape)

(110144, 6)
(27537, 6)


In [ ]:
#categorical_data = real_diameter_df.select_dtypes(include=['object']).copy()
new_columns = pd.get_dummies(categorical_data, columns=categorical_data.columns, drop_first=True)
new_columns_test = pd.get_dummies(categorical_data_test, columns=categorical_data_test.columns, drop_first=True)
new_columns

,Orbit condition code_1,Orbit condition code_2,Orbit condition code_3.0,Orbit condition code_4.0,Orbit condition code_5.0,Orbit condition code_6.0,Orbit condition code_7.0,Orbit condition code_8.0,Orbit condition code_9.0,Orbit condition code_0,...,spec_T_XDC,spec_T_XF,spec_T_XFC,spec_T_XFCU,spec_T_XFU,spec_T_XSCU,spec_T_XU,spec_T_Z,neo_Y,pha_Y
115463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241880,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
222606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313623,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
new_columns, new_columns_test = new_columns.align(new_columns_test, join='left', axis=1)

In [ ]:
new_columns = new_columns.fillna(0)
new_columns_test = new_columns_test.fillna(0)

In [ ]:
#real_diameter_df = real_diameter_df.drop(['neo' , 'pha'], axis = 1)

In [ ]:
dtf_train = dtf_train.drop(categorical_data.columns, axis = 1)
dtf_test = dtf_test.drop(categorical_data_test.columns, axis = 1)

In [ ]:
model_df_train = pd.concat([dtf_train, new_columns], axis=1)
model_df_test = pd.concat([dtf_test, new_columns_test], axis=1)
model_df_train.head()

,semi-major axis,eccentricity,Magnitude slope parameter,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,data arc-span(d),...,spec_T_XD,spec_T_XD:,spec_T_XDC,spec_T_XF,spec_T_XFC,spec_T_XFCU,spec_T_XFU,spec_T_XSCU,spec_T_XU,spec_T_Z
115463,2.293673,0.163461,0.1871,2.441385,334.870064,66.302762,1.918747,2.668598,3.473804,8233.0,...,0,0,0,0,0,0,0,0,0,0
24218,2.207402,0.175629,0.1871,1.874951,242.421340,30.671744,1.819720,2.595085,3.279672,23289.0,...,0,0,0,0,0,0,0,0,0,0
459444,3.192089,0.227003,0.1871,26.874278,65.645193,283.338537,2.467473,3.916704,5.703226,7264.0,...,0,0,0,0,0,0,0,0,0,0
241880,3.199675,0.153845,0.1871,25.573412,215.534567,48.627799,2.707420,3.691930,5.723569,6235.0,...,0,0,0,0,0,0,0,0,0,0
53921,2.596313,0.243720,0.1871,2.657834,160.847473,198.996805,1.963540,3.229087,4.183540,9741.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
new_columns.shape

(110144, 185)

In [ ]:
model_df_test.isna().sum()


semi-major axis                    0
eccentricity                       0
Magnitude slope parameter          0
Inclination with ecliptic plane    0
Longitude of the ascending node    0
                                  ..
spec_T_XFCU                        0
spec_T_XFU                         0
spec_T_XSCU                        0
spec_T_XU                          0
spec_T_Z                           0
Length: 205, dtype: int64

In [ ]:
model_df_train.shape

(110144, 205)

In [ ]:
model_df_test.shape

(27537, 205)

In [ ]:
model_df_train.dtypes

semi-major axis                    float64
eccentricity                       float64
Magnitude slope parameter          float64
Inclination with ecliptic plane    float64
Longitude of the ascending node    float64
                                    ...   
spec_T_XFCU                          uint8
spec_T_XFU                           uint8
spec_T_XSCU                          uint8
spec_T_XU                            uint8
spec_T_Z                             uint8
Length: 205, dtype: object

# Asteroids


*   **imbalanced data**



In [ ]:
# plt.figure(figsize=(20,17))
# cor = model_df_train.corr()
# sns.heatmap(model_df_train.corr(method='pearson'),annot=True)
# plt.title("Correlation Between Features")
# plt.show()

In [ ]:
model_df_train['diameter'].value_counts()

3.008     42
3.126     40
3.122     39
4.017     38
3.296     37
          ..
14.286     1
11.618     1
1.054      1
15.752     1
14.900     1
Name: diameter, Length: 15296, dtype: int64

## Linear Regression

In [ ]:
y_train = model_df_train['diameter']
X_train = model_df_train.drop(['diameter'], axis = 1)
y_test = model_df_test['diameter']
X_test = model_df_test.drop(['diameter'], axis = 1)

In [ ]:
X_test.shape

(27537, 204)

In [ ]:
X_train.isna().sum().sum()

0

In [ ]:
multi_reg = LinearRegression()
multi_reg.fit(X_train , y_train)
pred = multi_reg.predict(X_test)

In [ ]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.7906855479919159
Test Accuracy :  0.5443139153623295


## Random Forest Regression

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.7906855479919159
Test Accuracy :  0.5443139153623295
